In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import io

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/slack/2020/20204' #subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/slack*' # all directories

In [3]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    try:
        for div in soup.find_all('div', {'class':'flex_column'}):
            status = div.find('p', {'class':'tiny'})
            result.append(' '.join(status.text.split()))
    except:
        result.append('?')
    
    return result

In [4]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item and 'rss' not in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            ts = item.split('/')[0]
            data = extract_html_data(html)
            data.insert(0, ts)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [5]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
results = pool.map(read_file, [file for file in files])
pool.close()
end_time = time.time()

In [6]:
print(end_time - start_time)

1.854912519454956


In [7]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)
print(len(res))

1835


In [1]:
column_names = ['source','service','timestamp','Login/SSO','Connections','Messaging','Link Previews','Posts/Files','Notifications','Calls','Search','Apps/Integrations/APIs','Workspace/Org Administration','?1','?2']
df = pd.DataFrame(res, columns=column_names)
df.pop('?1')
df.pop('?2')
df.head(50)

NameError: name 'pd' is not defined

In [10]:
df.to_csv(r'/home/shane/Documents/thesis/output/parsed/final/slack.csv', index=False)